# Read and clean a single file

In [115]:
import os
import sys
import logging
import pandas as pd

from pandas_profiling import ProfileReport


project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))

from data_manipulator import get_input_data

bucket = 'yuma_bucket'
directory = 'BigData'

## Read Metadata
-----
General info for each relevant column

In [116]:
file = 'gs://{}/{}/data/metadatos-llamadas-urg-y-emer.csv'.format(bucket, directory)
df_metadata = pd.read_csv(file, encoding='latin1', sep=';')
df_metadata

,NOMBRE,DESCRIPCION
0,FECHA_INCIDENTE,Es la fecha el cual se registra la llamada del...
1,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,Es la fecha el cual se inicia el desplazamient...
2,CODIGO LOCALIDAD,Es el código de las 20 localidades de la ciuda...
3,LOCALIDAD,Es la localidad donde sucede el incidente.
4,EDAD,La edad del paciente.
5,UNIDAD,"La descripción de la edad si es en horas, días..."
6,GENERO,Es la distinción de genero del paciente
7,RED,Es la localización a nivel bogota de la red de...
8,TIPO_INCIDENTE,Es la descripción inicial que tipifica el cent...
9,PRIORIDAD,Es la tipificación según la prioridad del inci...


The final data must contains this and only this columns with that specific headers in order to standarize the data, this is a type of data dictionary but it also must include the data types (**schema**) of each field (**column**)

## Get raw data

In [117]:
raw_data_sep = get_input_data(bucket=bucket,
                          initial_directory=directory,
                          filename = 'datos-abiertos-septiembre-2019.csv')
print(raw_data_sep.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17018 entries, 0 to 17017
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   17018 non-null  int64 
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  17018 non-null  object
 2   CODIGO DE LOCALIDAD                17018 non-null  int64 
 3   LOCALIDAD                          17018 non-null  object
 4   EDAD                               17018 non-null  int64 
 5   UNIDAD                             9371 non-null   object
 6   GENERO                             17018 non-null  object
 7   RED                                17018 non-null  object
 8   TIPO_INCIDENTE                     17018 non-null  object
 9   PRIORIDAD                          17018 non-null  object
dtypes: int64(3), object(7)
memory usage: 1.3+ MB
None


It seems that this file contains already the fields, but need to rename them

### Rename Columns

In [118]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data_sep.rename(
    columns = {
        'NUMERO_INCIDENTE'                  :'FECHA_INCIDENTE',
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
    },
    inplace=True
)
raw_data_sep.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,538006196,2019-09-01 00:05:32,18,Rafael Uribe Uribe,53,Años,MASCULINO,Sur,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
1,538024196,2019-09-01 00:12:07,11,Suba,0,NaN,SIN_DATO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
2,538031196,2019-09-01 00:05:39,8,Kennedy,26,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
3,538084196,2019-09-01 00:13:39,11,Suba,24,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
4,538123196,2019-09-01 00:20:31,2,Chapinero,0,NaN,SIN_DATO,Norte,INCONSCIENTE/PARO CARDIORRESPIRATORIO,ALTA


### Check values per field
--------
1. FECHA_INICIO_DESPLAZAMIENTO_MOVIL
2. LOCALIDAD
3. UNIDAD
4. GENERO
5. RED
6. TIPO_INCIDENTE
7. PRIORIDAD

In [119]:
col = 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL'
raw_data_sep[col] = pd.to_datetime(raw_data_sep[col], format='%Y/%m/%d %H:%M') 
raw_data_sep[col].value_counts
raw_data_sep.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,538006196,2019-09-01 00:05:32,18,Rafael Uribe Uribe,53,Años,MASCULINO,Sur,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
1,538024196,2019-09-01 00:12:07,11,Suba,0,NaN,SIN_DATO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
2,538031196,2019-09-01 00:05:39,8,Kennedy,26,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
3,538084196,2019-09-01 00:13:39,11,Suba,24,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
4,538123196,2019-09-01 00:20:31,2,Chapinero,0,NaN,SIN_DATO,Norte,INCONSCIENTE/PARO CARDIORRESPIRATORIO,ALTA


In [121]:
col = 'LOCALIDAD'
raw_data_sep[col] = raw_data_sep[col].apply(lambda x: x.strip())
raw_data_sep[col].value_counts(dropna=False)

Kennedy               2398
Suba                  1634
Engativa              1631
Bosa                  1294
Ciudad Bolivar        1138
Puente Aranda          938
Rafael Uribe Uribe     854
Usaquen                829
San Cristobal          826
Fontibon               809
Chapinero              749
Barrios Unidos         616
Tunjuelito             603
Los Martires           593
Usme                   572
Santa Fe               549
Teusaquillo            540
Antonio Nariño         362
La Candelaria           83
Name: LOCALIDAD, dtype: int64

It seems that in some fields exist a white space at the begining of the values, probably at the end too, lets clean them all

In [122]:
raw_data_sep.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,538006196,2019-09-01 00:05:32,18,Rafael Uribe Uribe,53,Años,MASCULINO,Sur,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
1,538024196,2019-09-01 00:12:07,11,Suba,0,NaN,SIN_DATO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
2,538031196,2019-09-01 00:05:39,8,Kennedy,26,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
3,538084196,2019-09-01 00:13:39,11,Suba,24,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
4,538123196,2019-09-01 00:20:31,2,Chapinero,0,NaN,SIN_DATO,Norte,INCONSCIENTE/PARO CARDIORRESPIRATORIO,ALTA


In [126]:
raw_data_sep['UNIDAD'].value_counts(dropna=False)

Años     9311
NaN      7647
Meses      42
Dias       10
Horas       8
Name: UNIDAD, dtype: int64

In [127]:
raw_data_sep['UNIDAD'].fillna('SIN_DATO').value_counts()

Años        9311
SIN_DATO    7647
Meses         42
Dias          10
Horas          8
Name: UNIDAD, dtype: int64

In [131]:
col = 'GENERO'
raw_data_sep[col] = raw_data_sep[col].apply(lambda x: x.strip())
raw_data_sep[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

SIN_DATO     7632
MASCULINO    5082
FEMENINO     4304
Name: GENERO, dtype: int64

In [132]:
col = 'RED'
raw_data_sep[col] = raw_data_sep[col].apply(lambda x: x.strip())
raw_data_sep[col].value_counts(dropna=False)

Norte      7343
Sur        6360
Privada    3315
Name: RED, dtype: int64

In [133]:
col = 'TIPO_INCIDENTE'
raw_data_sep[col] = raw_data_sep[col].apply(lambda x: x.strip())
raw_data_sep[col].value_counts(dropna=False)

ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS    6333
INCONSCIENTE/PARO CARDIORRESPIRATORIO        1720
DIFICULTAD RESPIRATORIA                      1386
ENFERMO                                      1137
CONVULSIONES                                 1039
HERIDOS                                       834
INTENTO DE SUICIDIO                           653
DOLOR TORÁCICO                                629
TRASTORNO MENTAL                              621
LESIONES PERSONALES                           593
PATOLOGÍA GINECO - OBSTÉTRICA                 421
SÍNTOMAS GASTROINTESTINALES                   360
ACCIDENTE CEREBRO VASCULAR                    352
CAÍDA                                         334
INTOXICACIONES                                156
IDEAS DE SUICIDIO                             113
VIOLENCIA SEXUAL                               48
INCENDIO ESTRUCTURAL                           40
SIN_DATO                                       32
SANGRADO VAGINAL                               25


In [134]:
col = 'PRIORIDAD'
raw_data_sep[col] = raw_data_sep[col].apply(lambda x: x.strip())
raw_data_sep[col].value_counts(dropna=False)

ALTA        14243
MEDIA        2550
BAJA          204
SIN_D          15
SIN_DATO        6
Name: PRIORIDAD, dtype: int64

# Profile of the Data
------
first install pandas profiling, from a terminal

````
pip install pandas-profiling
````

In [23]:
project_dir

'/home/jupyter/ESEIT_BigData'

In [135]:
profile = ProfileReport(df=raw_data_sep)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_septiembre_2019.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Form the initial report of the file we see that are duplicates rows in the table, we need to subtract them

### Remove duplicates

In [136]:
print('Number of rows before cleaning:', raw_data_sep.shape[0] )
raw_data_sep.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data_sep.shape[0] )

Number of rows before cleaning: 17018
Number of rows after cleaning: 16267


### Save the final table
use to_csv to save the table

In [137]:
# saving in a local directory
raw_data_sep.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_septiembre_2019.csv', encoding='latin1', sep=';', index=False)